Провести процедуру A/B-тестирования.
1. Найти данные в открытых источниках или сгенерировать самостоятельно. Вы вольны сами задать условия эксперимента, это может быть что угодно, например тест нового сорта пива в баре на тестовой группе людей, изменение элементов дизайна сайта, введение повышенного кешбэка в банке, создание прямого рейса между городами, где ранее было доступны только рейсы с пересадками.
2. Продумайте дизайн вашего эксперимента: как разделить объекты на группы, как соблюсти соответствие условий эксперимента реальным условиям использования вашего продукта/сервиса.
3. Выберите метрику, на которую должно повлиять ваше воздействие.
4. Для вашей метрики используйте подходящий статистический критерий. Выберите и обоснуйте минимальный размер эффекта, который вы хотите задетектировать, уровни ошибки 1го и 2го рода, на которые вы согласны.
5. Оцените, сколько наблюдений вам потребуется, чтобы провести данный эксперимент.
6. Проведите AA-тест, чтобы проверить дизайн эксперимента.
7. Затем проведите AB-тест.
8. Какие выводы можно сделать на основе вашего эксперимента?

### 1-3. Дизайн эксперимента

Зададим экспреимент. Пусть у нас есть сайт, маркетплейс, на котором продаются товары для дома. Компания стремится увеличить лояльность пользователей и повысить свой авторитет на рынке. Даже готова отказаться от рекламы на маркетплейсе, если это конечно повлечет за собой повышение лояльности. В связи с этим возникает необходимость понять, будет ли отсутствие рекламы на сайте привлекать больше пользователей - вернутся ли пользователи на сайт спустя неделю. Итак, **метрикой** в нашем эксперементе будет **возвращаемость пользователя (retention)**.

Скажем, что у каждого пользователя нашего сайта есть числовой идентификатор (индекс в базе данных всех пользователей). Разделим пользователей на 2 группы: A) пользователи с четным идентификатором; B) с нечетным. Таким образом, мы исключим зависимости данных от разбиения, ведь такое разбиение никак не влияет на то, вернется пользователь на сайт или нет.

Как вы понимаете, в открытых источниках таких данных не найти, так что будем производить генерацию. Будем проверять гипотезу о том, что доли вернувшихся пользователей совпадают. В качестве альтернативы рассматриваем случаи, когда из-за отключения рекламы пользователи возвращаться на сайт стали чаще.

$H_0: p_0 = p_a$, где $p_0$ - доля вернувшихся на сайт через неделю пользователей со включенной рекламой, $p_a$ - с отключенной

$H_1: p_0 < p_a$

### 4-5. Критерий, минимальный размер эффекта, ошибки. Число наблюдений.

Поговорим теперь об ошибка 1-го и 2-го рода.

При ошибке 1-го рода у нас верна $H_0$, т.е. отключение рекламы ни на что не влияет, но мы отвергаем эту гипотезу, т.е. считаем, что отключение рекламы нам интересно. Если мы отключим рекламу, то мы просто потеряем часть дохода, а пользователей возвращаться больше не станет - лояльность не приобретем. В таком случае нам очень бы не хотелось ошибаться, так что будем строго оценивать такие ошибки и возьмем $\alpha = 0.01$

При ошибке 2-го рода верна $H_a$, т.е. отключение рекламы повлечет за собой бОльшее количество вернувшихся пользователей, но мы принимаем $H_0$, т.е. считаем, что отключение рекламы не имеет смысла. Таким образом, здесь мы возможно недополучим лояльность пользователей, однако у нас останется реклама, которая будет приносить доход. Такую ошибку оценим менее строго чем прошлую и возьмем $\beta = 0.05$

Говоря о минимальном детектируемом размере эффекта ($p_a - p_0$), учтем, что отключение рекламы влечет за собой отсутствие дохода с этой рекламы. Т.е. если мы ее отключаем, мы хотим, чтобы это было не безвозмездно - значительно больше пользователей возвращалось на сайт $⇒$ рассмотрим минимальный эффект в 0.05

В качестве статистического критерия будем использовать Z-критерий для разности независимых долей, который как раз работает с распределением Бернулли и применяется для двух выборок.


In [1]:
import numpy as np
from scipy import stats

In [2]:
def min_n(p0, pa, alpha, beta):
    za = stats.norm.ppf(1 - alpha)
    zb = stats.norm.ppf(1 - beta)
    n = za * np.sqrt(p0*(1 - p0)) + zb * np.sqrt(pa*(1-pa))
    n /= (pa - p0)
    return int(np.ceil(n*n))

p0 = 0.1
pa = 0.15
alpha = 0.01
beta = 0.05

n = min_n(p0, pa, alpha, beta)
print("Необходимое для проведения эксперимента число наблюдений =", n)

Необходимое для проведения эксперимента число наблюдений = 661


In [3]:
np.random.seed(15)
group_A = { "No changes" : np.random.binomial(1, p0, n), "Change" : np.random.binomial(1, p0, n) }
group_B = { "No changes" : np.random.binomial(1, p0, n), "Change" : np.random.binomial(1, pa, n) }

### 6. AA-тест

In [7]:
from statsmodels.stats.proportion import proportions_ztest

count = np.array([np.sum(group_A["No changes"]), np.sum(group_B["No changes"])])
nobs = np.array([n, n])
stat, p_value = proportions_ztest(count, nobs, alternative='smaller')
stat, p_value

(-1.3207010151078933, 0.09330053774215835)

После прохождения AA теста посмотрим на результаты AB-теста

### 7. AB-тест

In [8]:
count = np.array([np.sum(group_A["Change"]), np.sum(group_B["Change"])])
nobs = np.array([n, n])
stat, p_value = proportions_ztest(count, nobs, alternative='smaller')
stat, p_value

(-2.0850658295420867, 0.01853166273046103)

In [9]:
p_value > alpha

True

### 8. Выводы

Для AB-теста гипотеза  $H_0$  не отвергается, т.к. p_value > α. Таким образом, при данном дизайне эксперимента отключение рекламы для маркетплейса не повлечет за собой значительное (> 5%) увеличение лояльности пользователей. Видимо, предположение о том, что показ рекламы на сайте негативно отражается на лояльности пользователей было ложно. Это в целом логично, ведь люди настолько уже привыкли к огромному количеству рекламы.